# 第5章：Taproot：Bitcoin 脚本系统的演化

> **参考**: `book/translations/zh-Hans/Chapter 5.md`  
> **代码示例**: `code/chapter05/`  
> **最后更新**: 2025-12-06

---

Taproot 代表了 Bitcoin 脚本演化的顶点，展示了最复杂的智能合约如何看起来与简单支付完全相同。这种革命性方法将 Schnorr 签名与密码学密钥调整（key tweaking）相结合，创建 Bitcoin 最先进和私密的授权系统。


## Taproot 承诺：统一隐私

Taproot 的根本突破是**支付统一性**。无论交易代表：
- 简单的单签名支付
- 复杂的多方合约
- Lightning Network 通道
- 具有多个授权级别的企业资金库

它们在花费之前在区块链上看起来完全相同。这种统一性由两个数学创新实现：Schnorr 签名和密钥调整。


## Schnorr 签名：数学基础

在理解 Taproot 架构之前，我们需要掌握使一切成为可能的数学优雅：Schnorr 签名及其变革属性，这些属性革命性地改变了 Bitcoin 的授权系统。

### 为什么选择 Schnorr？ECDSA 的局限性

Bitcoin 最初使用 ECDSA（椭圆曲线数字签名算法）进行数字签名，但这一选择带来了 Schnorr 完全消除的重大局限性：

**ECDSA 问题：**
- **可延展性**：签名可以在不使其失效的情况下被修改
- **无法聚合**：多个签名无法组合
- **更大尺寸**：签名通常为 71-72 字节
- **复杂验证**：需要更多计算资源
- **无线性性**：数学运算不保持关系

**Schnorr 的革命性优势：**
- **不可篡改**：在 BIP340 下，确定性 nonce、x-only 公钥和严格编码规则消除了 ECDSA 中看到的第三方可延展性向量
- **密钥聚合**：多个公钥可以组合为一个
- **单签名输出**：产生单个聚合签名
- **紧凑尺寸**：固定 64 字节签名
- **高效验证**：更快更简单的验证过程
- **数学线性性**：支持高级密码学构造

### 改变游戏规则的属性：线性性

使 Taproot 成为可能的数学突破是 Schnorr 的**线性性属性**：

```
If Alice has signature A for message M
And Bob has signature B for the same message M  
Then A + B creates a valid signature for (Alice + Bob)'s combined key
```

这个简单的数学关系支持三个革命性能力：

1. **密钥聚合**：多个人可以将他们的公钥组合为一个
2. **单签名输出**：多方可以协作产生一个统一的签名
3. **密钥调整**：密钥可以通过承诺确定性修改

注意："单签名输出"指的是通过 MuSig2（钱包级协议）在链上产生一个 BIP340 签名，而不是跨输入的共识级签名聚合。

### 视觉对比：ECDSA vs Schnorr

```
ECDSA Multisig (3-of-3):
┌─────────────────────────────────────┐
│           Transaction               │
├─────────────────────────────────────┤
│ Alice Signature:   [71 bytes]       │
│ Bob Signature:     [72 bytes]       │
│ Charlie Signature: [70 bytes]       │
├─────────────────────────────────────┤
│ Total Size: ~213 bytes              │
│ Verifications: 3 separate           │
│ Privacy: REVEALS 3 participants     │
│ Appearance: 👥👥👥 (obviously multi) │
└─────────────────────────────────────┘

Schnorr Aggregated (3-of-3):
┌─────────────────────────────────────┐
│           Transaction               │
├─────────────────────────────────────┤
│ Aggregated Signature: [64 bytes]    │
├─────────────────────────────────────┤
│ Total Size: 64 bytes                │
│ Verifications: 1 single check       │
│ Privacy: REVEALS nothing about #    │
│ Appearance: 👤 (looks like single)  │
└─────────────────────────────────────┘
```

**隐私魔法：**
```
External Observer sees:
┌─────────────────┬─────────────────┐
│   Transaction A │   Transaction B │
├─────────────────┼─────────────────┤
│ 64-byte signature│ 64-byte signature│
│ Looks like: 👤  │ Looks like: 👤  │
└─────────────────┴─────────────────┘

Reality:
┌─────────────────┬─────────────────┐
│   Transaction A │   Transaction B │
├─────────────────┼─────────────────┤
│ Actually: 👤    │ Actually: 👥👥👥 │
│ (1 person)      │ (3 people)      │
└─────────────────┴─────────────────┘

🔮 Impossible to distinguish from outside!
```


## 密钥调整：通往 Taproot 的桥梁

Taproot 通过**密钥调整（key tweaking）**（在 BIP340/341/342 哲学中也称为**可调整承诺（tweakable commitment）**）利用 Schnorr 的线性性。

概念上： 
```
t = H("TapTweak" || internal_pubkey || merkle_root)
```
形式上（BIP341）：

```
t  = int(HashTapTweak(xonly_internal_key || merkle_root_or_empty)) mod n

P' = P + t * G
d' = d + t
```
**Even-Y 要求（BIP340）：**  
Taproot 使用 x-only 公钥——但 secp256k1 上的实际点仍然有两个可能的 y 值（偶数/奇数）。  
BIP340 规则是：最终调整的输出密钥**必须对应于 even-y 点**。  
如果点最终是 odd-y，实现将私钥翻转为 `d' = n − d'`，使得 `P' = d'*G` 落在偶数分支上。

（为什么这很重要：在 script-path 支出中，这个奇偶性被编码到控制块的最低位。如果现在不跟踪这个，script-path 以后将无法验证。）

### 密钥调整结构的视觉表示

```
Internal Key (P) ─────────► + tweak ─────────► Output Key (P')
                              ▲                      │
                              │                      │
                       Merkle Root ◄────────────────┘
                    script_path_commitment
```

**密钥关系图：**
```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│   Internal Key  │    │   Tweak Value   │    │   Output Key    │
│       (P)       │    │   t = H(P||M)   │    │      (P')       │
│                 │───►│                 │───►│                 │
│ User's original │    │ Deterministic   │    │ Final address   │
│ private key     │    │ from commit     │    │ seen on chain   │
└─────────────────┘    └─────────────────┘    └─────────────────┘
        │                        ▲                        │
        │                        │                        │
        └─── Can compute d' ─────┘                        │
                                                          │
                                 ┌─────────────────────────┘
                                 │
                                 ▼
                      ┌─────────────────┐
                      │   Merkle Root   │
                      │       (M)       │
                      │                 │
                      │ Commitment to   │
                      │ all possible    │
                      │ spending paths  │
                      └─────────────────┘
```

其中：
- `P` = **内部密钥（Internal Key）**（原始公钥，用户控制）
- `M` = **Merkle 根（Merkle Root）**（对所有可能花费条件的承诺）
- `t` = **调整值（Tweak Value）**（从 P 和 M 确定性计算）
- `P'` = **输出密钥（Output Key）**（最终 Taproot 地址，出现在区块链上）
- `d'` = **调整后的私钥（Tweaked Private Key）**（用于 key-path 支出）

这种数学关系确保：
1. **任何人都可以从 P 和承诺计算 P'**（给定内部密钥 P 和（可选）Merkle 根 M）
2. **只有密钥持有者可以从 d 和调整值计算 d'**
3. **关系 d' × G = P'** 得到保持（签名验证有效）

### 实际密钥调整实现


In [ ]:
# 示例 1: 密钥调整演示
# 参考: code/chapter05/01_demonstrate_key_tweaking.py

from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey
import hashlib

def demonstrate_key_tweaking():
    """Demonstrates the complete key tweaking process"""
    setup('testnet')
    
    # Step 1: Generate internal key pair
    internal_private_key = PrivateKey('cTALNpTpRbbxTCJ2A5Vq88UxT44w1PE2cYqiB3n4hRvzyCev1Wwo')
    internal_public_key = internal_private_key.get_public_key()
    
    print("=== STEP 1: Internal Key Generation ===")
    print(f"Internal Private Key: {internal_private_key.to_wif()}")
    print(f"Internal Public Key:  {internal_public_key.to_hex()}")
    
    # Step 2: Create simple script commitment (we'll use empty for this example)
    # In real Taproot, this would be a Merkle root of script conditions
    script_commitment = b''  # Empty = key-path-only spending
    
    print(f"\n=== STEP 2: Script Commitment ===")
    print(f"Script Commitment: {script_commitment.hex() if script_commitment else 'Empty (key-path-only)'}")
    
    # Step 3: Calculate tweak using BIP341 formula
    # Extract x-only public key (skip the 0x02 or 0x03 prefix)
    internal_pubkey_hex = internal_public_key.to_hex()
    internal_pubkey_bytes = bytes.fromhex(internal_pubkey_hex[2:])  # x-only (skip prefix)
    
    # BIP341 HashTapTweak: SHA256("TapTweak" || xonly_internal_key || merkle_root)
    tweak_preimage = b'TapTweak' + internal_pubkey_bytes + script_commitment
    tweak_hash = hashlib.sha256(tweak_preimage).digest()
    tweak_int = int.from_bytes(tweak_hash, 'big')
    
    print(f"\n=== STEP 3: Tweak Calculation ===")
    print(f"Formula: t = HashTapTweak(xonly_internal_key || merkle_root)")
    print(f"")
    print(f"Internal PubKey (x-only): {internal_pubkey_bytes.hex()}")
    print(f"Script Commitment: {script_commitment.hex() if script_commitment else '(empty)'}")
    print(f"")
    print(f"Tweak Preimage: TapTweak || {internal_pubkey_bytes.hex()} || {script_commitment.hex()}")
    print(f"Tweak Hash (SHA256): {tweak_hash.hex()}")
    print(f"Tweak Integer (t): {tweak_int}")
    print(f"")
    print(f"Note: The tweak value 't' is a 256-bit integer derived from")
    print(f"      the internal public key and script commitment.")
    
    # Step 4: Apply tweaking formula
    # d' = d + t (mod n)
    # P' = P + t×G = d'×G
    internal_privkey_int = int.from_bytes(internal_private_key.to_bytes(), 'big')
    curve_order = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
    tweaked_privkey_int = (internal_privkey_int + tweak_int) % curve_order
    
    # Create tweaked private key from the integer
    tweaked_private_key = PrivateKey.from_bytes(tweaked_privkey_int.to_bytes(32, 'big'))
    tweaked_public_key = tweaked_private_key.get_public_key()
    
    print(f"\n=== STEP 4: Tweaking Application ===")
    print(f"Formula for Private Key: d' = d + t (mod n)")
    print(f"Formula for Public Key:  P' = P + t×G = d'×G")
    print(f"")
    print(f"Where:")
    print(f"  d  = original private key (internal)")
    print(f"  d' = tweaked private key (output)")
    print(f"  t  = tweak value (from Step 3)")
    print(f"  P  = original public key (internal)")
    print(f"  P' = tweaked public key (output)")
    print(f"  G  = generator point on secp256k1 curve")
    print(f"  n  = curve order")
    print(f"")
    print(f"Private Key Transformation:")
    print(f"  Original (d):  {internal_privkey_int}")
    print(f"  Tweak (t):     +{tweak_int}")
    print(f"  ─────────────────────────────────────────────")
    print(f"  Tweaked (d'):  {tweaked_privkey_int}")
    print(f"  (mod n: {curve_order})")
    print(f"")
    print(f"Public Key Transformation:")
    print(f"  Original (P):  {internal_public_key.to_hex()}")
    print(f"  Tweaked (P'):  {tweaked_public_key.to_hex()}")
    print(f"  Output Key (x-only): {tweaked_public_key.to_hex()[2:]}")
    print(f"")
    print(f"Key Insight: P' = d'×G = (d + t)×G = d×G + t×G = P + t×G")
    
    # Step 5: Verify the mathematical relationship
    print(f"\n=== STEP 5: Mathematical Verification ===")
    verification_result = tweaked_private_key.get_public_key().to_hex() == tweaked_public_key.to_hex()
    print(f"Verification: d' × G = P'? {verification_result}")
    print(f"")
    print(f"This confirms:")
    print(f"  ✓ The tweaked private key d' correctly generates the tweaked public key P'")
    print(f"  ✓ The relationship P' = P + t×G holds mathematically")
    print(f"  ✓ Anyone can compute P' from P and the commitment (public information)")
    print(f"  ✓ Only the key holder can compute d' from d and tweak (private information)")
    
    return {
        'internal_private': internal_private_key,
        'internal_public': internal_public_key,
        'tweak_hash': tweak_hash,
        'tweaked_private': tweaked_private_key,
        'tweaked_public': tweaked_public_key
    }


if __name__ == "__main__":
    result = demonstrate_key_tweaking()
    
    print("\n" + "=" * 70)
    print("SUMMARY: THE KEY TWEAKING PROCESS")
    print("=" * 70)
    print("1. Start with internal key pair (d, P)")
    print("2. Calculate tweak: t = HashTapTweak(xonly_P || merkle_root)")
    print("3. Apply tweaking:")
    print("   - Private key: d' = d + t (mod n)")
    print("   - Public key:  P' = P + t×G = d'×G")
    print("4. Result: Output key (P') that commits to script conditions")
    print("")
    print("=" * 70)
    print("KEY INSIGHTS FROM KEY TWEAKING")
    print("=" * 70)
    print("1. Dual Spending Paths:")
    print("   - Key Path: Use the tweaked private key (d') to sign directly (cooperative)")
    print("   - Script Path: Reveal the internal public key (P) and prove script execution (fallback)")
    print("")
    print("2. Cryptographic Binding:")
    print("   - The tweak (t) cryptographically binds the output key (P') to specific script commitments")
    print("   - Changing the commitment changes the tweak, which changes the output key")
    print("")
    print("3. Deterministic Verification:")
    print("   - Anyone can verify that a tweaked key correctly commits to specific conditions")
    print("   - Given P and merkle_root, anyone can compute P' and verify it matches")
    print("")
    print("4. Privacy Through Indistinguishability:")
    print("   - The tweaked public key (P') is mathematically indistinguishable from any other Schnorr public key")
    print("   - Simple payments and complex contracts look identical until spent")


**密钥调整的关键洞察：**

1. **双重花费路径**：调整后的密钥创建两种花费方法：
   - **Key Path**：使用调整后的私钥直接签名（协作）
   - **Script Path**：揭示内部公钥并证明脚本执行（回退）

2. **密码学绑定**：调整值将输出密钥密码学绑定到特定脚本承诺

3. **确定性验证**：任何人都可以验证调整后的密钥正确承诺到特定条件

4. **通过不可区分性实现隐私**：调整后的公钥在数学上与任何其他 Schnorr 公钥不可区分


## 为什么这能实现统一外观

Schnorr 签名和密钥调整的结合创造了"统一外观"魔法：

```
Simple Payment:
├── Internal Key: Just a regular private key
├── Script Commitment: Empty (no conditions)
├── Tweaked Key: Internal key + H(key || empty)
└── Spending: 64-byte Schnorr signature

Complex Contract:
├── Internal Key: Same regular private key
├── Script Commitment: Merkle root of 100 conditions
├── Tweaked Key: Internal key + H(key || merkle_root)
└── Spending: 64-byte Schnorr signature (if cooperative)

🔍 External View: IDENTICAL 64-byte signatures!
```


## 简单 Taproot 交易：整合所有内容

现在让我们通过基本的 Taproot 到 Taproot 交易看看这在实践中如何工作：


In [ ]:
# 示例 2: 创建简单 Taproot 交易
# 参考: code/chapter05/02_create_simple_taproot_transaction.py

from bitcoinutils.setup import setup
from bitcoinutils.utils import to_satoshis
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
from bitcoinutils.keys import PrivateKey, P2trAddress

def create_simple_taproot_transaction():
    """Creates a basic Taproot-to-Taproot transaction"""
    setup('testnet')
    
    # Sender's information
    from_private_key = PrivateKey('cPeon9fBsW2BxwJTALj3hGzh9vm8C52Uqsce7MzXGS1iFJkPF4AT')
    from_pub = from_private_key.get_public_key()
    from_address = from_pub.get_taproot_address()
    
    # Receiver's address
    to_address = P2trAddress('tb1p53ncq9ytax924ps66z6al3wfhy6a29w8h6xfu27xem06t98zkmvsakd43h')
    
    print("=== TAPROOT TRANSACTION CREATION ===")
    print(f"From Address: {from_address.to_string()}")
    print(f"To Address: {to_address.to_string()}")
    
    print(f"\n=== TRANSACTION SETUP ===")
    print(f"Sender Private Key: {from_private_key.to_wif()}")
    print(f"Sender Public Key:  {from_pub.to_hex()}")
    print(f"From Address (P2TR): {from_address.to_string()}")
    print(f"To Address (P2TR):   {to_address.to_string()}")
    
    # Create transaction input
    txin = TxInput(
        'b0f49d2f30f80678c6053af09f0611420aacf20105598330cb3f0ccb8ac7d7f0',
        0
    )
    
    print(f"\n=== INPUT DETAILS ===")
    print(f"Previous TXID: b0f49d2f30f80678c6053af09f0611420aacf20105598330cb3f0ccb8ac7d7f0")
    print(f"VOUT: 0")
    
    # Input amount and script for signing
    input_amount = 0.00029200
    amounts = [to_satoshis(input_amount)]
    input_script = from_address.to_script_pub_key()
    scripts = [input_script]
    
    print(f"Input Amount: {input_amount} BTC ({to_satoshis(input_amount)} satoshis)")
    print(f"Input ScriptPubKey: {input_script.to_hex()}")
    print(f"  Format: OP_1 <32-byte-output-key> (Taproot)")
    
    # Create transaction output
    amount_to_send = 0.00029000
    txout = TxOutput(
        to_satoshis(amount_to_send),
        to_address.to_script_pub_key()
    )
    
    print(f"\n=== OUTPUT DETAILS ===")
    print(f"Output Amount: {amount_to_send} BTC ({to_satoshis(amount_to_send)} satoshis)")
    print(f"Output ScriptPubKey: {to_address.to_script_pub_key().to_hex()}")
    print(f"  Format: OP_1 <32-byte-output-key> (Taproot)")
    print(f"Fee: {input_amount - amount_to_send} BTC ({to_satoshis(input_amount - amount_to_send)} satoshis)")
    
    # Create transaction with SegWit enabled
    tx = Transaction([txin], [txout], has_segwit=True)
    
    print(f"\n=== UNSIGNED TRANSACTION ===")
    unsigned_tx = tx.serialize()
    print(f"Hex: {unsigned_tx}")
    print(f"TxId: {tx.get_txid()}")
    print(f"Size: {len(unsigned_tx) // 2} bytes")
    
    # Sign the transaction using Schnorr signature
    # The sign_taproot_input() API handles the complex sighash construction:
    # 1. Builds BIP341 sighash with all input amounts and scripts
    # 2. Creates the signature message: sighash + key_version + code_separator
    # 3. Generates 64-byte Schnorr signature using tweaked private key
    print(f"\n=== SIGNING PROCESS ===")
    print(f"Signing with Schnorr signature (BIP340)...")
    print(f"  - Building BIP341 sighash with input amounts and scripts")
    print(f"  - Creating signature message: sighash + key_version + code_separator")
    print(f"  - Generating 64-byte Schnorr signature using tweaked private key")
    
    sig = from_private_key.sign_taproot_input(
        tx,
        0,
        scripts,
        amounts
    )
    
    print(f"\n=== SIGNATURE DETAILS ===")
    sig_bytes = len(sig) // 2  # Hex string length / 2 = bytes
    print(f"Signature Length: {sig_bytes} bytes (hex: {len(sig)} chars)")
    print(f"Signature (hex): {sig}")
    print(f"  r-value: {sig[:64]} (32 bytes)")
    print(f"  s-value: {sig[64:]} (32 bytes)")
    print(f"  Total: {sig_bytes} bytes (fixed size, no variable encoding)")
    print(f"  Note: This is a BIP340 Schnorr signature (64 bytes)")
    
    # Add witness data - the simplification here reflects Taproot's efficiency
    # Unlike SegWit's [signature, public_key], Taproot needs only [signature]
    # The public key is embedded in the scriptPubKey as the output key
    print(f"\n=== WITNESS CONSTRUCTION ===")
    print(f"Taproot Witness Structure:")
    print(f"  - Unlike SegWit P2WPKH: [signature, public_key]")
    print(f"  - Taproot P2TR: [signature] only")
    print(f"  - Public key is embedded in scriptPubKey as output key")
    print(f"  - Witness items: 1 (just the signature)")
    
    tx.witnesses.append(TxWitnessInput([sig]))
    
    # Get signed transaction
    signed_tx = tx.serialize()
    
    print(f"\n=== SIGNED TRANSACTION ===")
    print(f"Hex: {signed_tx}")
    print(f"Size: {len(signed_tx) // 2} bytes")
    print(f"Virtual Size: {tx.get_vsize()} vbytes")
    
    print(f"\n=== TRANSACTION SUMMARY ===")
    print(f"Send Amount: {amount_to_send} BTC ({to_satoshis(amount_to_send)} satoshis)")
    print(f"Fee: {input_amount - amount_to_send} BTC ({to_satoshis(input_amount - amount_to_send)} satoshis)")
    print(f"Transaction Size: {tx.get_size()} bytes")
    print(f"Virtual Size: {tx.get_vsize()} vbytes")
    
    print(f"\nKey Observations:")
    print(f"1. Taproot Address Generation: get_taproot_address() automatically applies the tweaking process")
    print(f"2. Schnorr Signing: sign_taproot_input() produces exactly 64-byte signatures")
    print(f"3. Minimal Witness: Only the signature is needed in the witness stack")
    print(f"   (In practice the item is 64 or 65 bytes: with SIGHASH_DEFAULT the 1-byte flag is omitted;")
    print(f"    for other sighashes the flag byte is appended)")
    print(f"4. Identical Appearance: This transaction looks identical to any other Taproot transaction")
    
    return tx, sig


if __name__ == "__main__":
    tx, signature = create_simple_taproot_transaction()
    
    print("\n" + "=" * 60)
    print("TAPROOT EFFICIENCY COMPARISON")
    print("=" * 60)
    print("Legacy P2PKH:")
    print("  - ScriptPubKey: OP_DUP OP_HASH160 <20-byte-hash> OP_EQUALVERIFY OP_CHECKSIG")
    print("  - ScriptSig: <signature> <public_key>")
    print("  - Size: ~225 bytes")
    print("  - Information Revealed: Single signature spending")
    print("\nSegWit P2WPKH:")
    print("  - ScriptPubKey: OP_0 <20-byte-hash>")
    print("  - Witness: [signature, public_key]")
    print("  - Size: ~165 bytes")
    print("  - Information Revealed: Single signature spending")
    print("\nTaproot P2TR:")
    print("  - ScriptPubKey: OP_1 <32-byte-output-key>")
    print("  - Witness: [schnorr_signature]")
    print("  - Size: ~135 bytes")
    print("  - Information Revealed: Nothing about internal complexity")
    print("\n🔮 The Magic: Simple and complex Taproot transactions are")
    print("   completely indistinguishable until spent!")


**关键观察：**

1. **Taproot 地址生成**：`get_taproot_address()` 自动应用调整过程
2. **Schnorr 签名**：`sign_taproot_input()` 产生恰好 64 字节的签名
3. **最小见证**：见证栈中只需要签名（实际上项是 64 或 65 字节：使用 SIGHASH_DEFAULT 时省略 1 字节标志；对于其他 sighash，标志字节被追加）
4. **相同外观**：此交易看起来与任何其他 Taproot 交易完全相同

## 真实交易分析

让我们检查一个真实的 Taproot 交易：`a3b4d0382efd189619d4f5bd598b6421e709649b87532d53aecdc76457a42cb6`

**交易结构：**
```
Input:
├── Previous Output: tb1pjyjeruun8pc5ln3wtv2d6lsxqn55frpyc83kn473h7848d0kj73sxy3ku8
├── ScriptPubKey: OP_1 912591f39338714fce2e5b14dd7e0604e9448c24c1e369d7d1bf8f53b5f697a3
└── Witness: [7d25fbc9b98ee0eb09ed38c2afc19127465b33d6120f4db8d4fd46e532e30450d7d2a1f1dd7f03e8488c434d10f4051741921d695a44fb774897020f41da99f3]

Output:
├── Destination: tb1p53ncq9ytax924ps66z6al3wfhy6a29w8h6xfu27xem06t98zkmvsakd43h
└── ScriptPubKey: OP_1 a3ff4d6e5ab7b6a8f5d5e8d6c7aef543782ba38c9e614d7e1a2b3c4f567890ab
```

**见证数据分析：**
```
Schnorr Signature: 7d25fbc9b98ee0eb09ed38c2afc19127465b33d6120f4db8d4fd46e532e30450d7d2a1f1dd7f03e8488c434d10f4051741921d695a44fb774897020f41da99f3

Structure:
├── r-value: 7d25fbc9b98ee0eb09ed38c2afc19127465b33d6120f4db8d4fd46e532e30450d
├── s-value: 7d2a1f1dd7f03e8488c434d10f4051741921d695a44fb774897020f41da99f3
└── Total: 64 bytes (32 + 32)
```

**关键洞察：**
- **固定长度**：恰好 64 字节，无可变编码
- **无公钥**：与 SegWit 不同，见证中无公钥
- **单一组件**：只有签名，最大效率

## Taproot 栈执行：Key Path 支出

让我们追踪 Taproot 交易的完整栈执行：

### 初始状态
交易以空栈开始：

```
│ (empty)                                 │
└─────────────────────────────────────────┘
```

### 1. OP_1：推送见证版本
scriptPubKey 以 OP_1 开始，表示这是版本 1 见证程序：

```
│ 01 (witness_version)                    │
└─────────────────────────────────────────┘
```

### 2. PUSH 输出密钥：加载 32 字节 Taproot 输出密钥
scriptPubKey 推送 32 字节输出密钥：

```
│ 912591f3...5f697a3 (output_key)         │
│ 01 (witness_version)                    │
└─────────────────────────────────────────┘
```

### 3. 模式识别：Bitcoin Core 检测 Taproot 格式
Bitcoin Core 识别模式 `OP_1 <32-bytes>` 并切换到 Taproot 执行模式：

**识别过程：**
1. **版本检查**：OP_1 = 见证版本 1
2. **长度检查**：32 字节 = Taproot 格式  
3. **执行切换**：加载 Taproot 解释器
4. **花费路径检测**：见证仅包含签名 = key path 支出

### 4. 加载见证：提取 Schnorr 签名
见证栈仅包含签名：

```
│ 7d25fbc9...da99f3 (schnorr_signature)   │
│ 912591f3...5f697a3 (output_key)         │
└─────────────────────────────────────────┘
```

### 5. Schnorr 验证：针对输出密钥验证签名
解释器执行 Schnorr 签名验证：

**验证算法：**
1. **解析签名**：提取 r 和 s 值（各 32 字节）
2. **计算挑战**：`e = H(r || P || sighash)`
3. **计算验证点**：`R = s×G - e×P`
4. **验证**：`r == x-coordinate of R`

**验证结果：**
```
│ 1 (TRUE)                                │
└─────────────────────────────────────────┘
```

**（交易有效——key path 支出成功）**

## 不可区分性的力量

Taproot 的革命性方面通过比较不同交易类型来展示：

### 视觉对比

```
Legacy P2PKH:
├── ScriptPubKey: OP_DUP OP_HASH160 <20-byte-hash> OP_EQUALVERIFY OP_CHECKSIG
├── ScriptSig: <signature> <public_key>
└── Size: ~225 bytes
   Information Revealed: Single signature spending

SegWit P2WPKH:
├── ScriptPubKey: OP_0 <20-byte-hash>
├── Witness: [signature, public_key]
└── Size: ~165 bytes
   Information Revealed: Single signature spending

Taproot P2TR (Simple):
├── ScriptPubKey: OP_1 <32-byte-output-key>
├── Witness: [schnorr_signature]
└── Size: ~135 bytes
   Information Revealed: Nothing about internal complexity

Taproot P2TR (Complex Contract):
├── ScriptPubKey: OP_1 <32-byte-output-key>
├── Witness: [schnorr_signature]
└── Size: ~135 bytes
   Information Revealed: Nothing about internal complexity
```

**魔法**：简单和复杂的 Taproot 交易在花费之前**完全不可区分**！


## 编程差异：从 SegWit 的演化

```python
# SegWit (P2WPKH) Pattern
def create_segwit_transaction():
    private_key = PrivateKey(...)
    address = private_key.get_segwit_address()  # P2WPKH
    
    # Signing
    signature = private_key.sign_segwit_input(tx, 0, script_code, amount)
    
    # Witness: [signature, public_key]
    tx.witnesses.append(TxWitnessInput([signature, public_key]))

# Taproot (P2TR) Pattern  
def create_taproot_transaction():
    private_key = PrivateKey(...)
    public_key = private_key.get_public_key()
    address = public_key.get_taproot_address()  # P2TR
    
    # Signing
    signature = private_key.sign_taproot_input(tx, 0, scripts, amounts)
    
    # Witness: [signature] - No public key needed!
    tx.witnesses.append(TxWitnessInput([signature]))
```

**关键编程变化：**
1. **地址生成**：必须首先获取公钥，然后获取 Taproot 地址
2. **签名方法**：`sign_taproot_input()` 使用 Schnorr 签名
3. **见证结构**：只需要签名，不需要公钥
4. **脚本格式**：使用数组表示脚本和金额


## 协作优势

Taproot 为协作创造强大激励：

```
Cooperative Spending (Key Path):
├── Parties: Alice, Bob, Charlie (all agree)
├── Witness: [64-byte signature]
├── Size: ~135 bytes
├── Privacy: Maximum (looks like single-sig)
└── Efficiency: Optimal

Non-Cooperative Spending (Script Path):
├── Parties: Alice, Bob, Charlie (dispute)
├── Witness: [script_data, revealed_script, control_block]
├── Size: ~200-500 bytes
├── Privacy: Partial (reveals one condition)
└── Efficiency: Reduced but still functional
```

**经济激励：**
- **协作奖励**：更小的费用，更好的隐私
- **冲突成本**：更大的交易，减少隐私
- **对齐**：技术优化与经济协作对齐


## 章节总结

Taproot 通过两个关键数学创新代表了 Bitcoin 交易的范式转变：

**Schnorr 签名**：线性性属性支持密钥聚合、单签名输出，最重要的是密钥调整。这创造了固定 64 字节签名，可以表示任何复杂程度，同时看起来完全相同。

**密钥调整（可调整承诺）**：数学关系 `P' = P + t×G` 允许密钥通过脚本承诺确定性修改，创建双重花费路径，同时保持密码学安全性。

**结果**：复杂的智能合约在计算和观察上与简单支付**完全相同**，在不牺牲功能的情况下提供前所未有的隐私。

**隐私革命**：所有 Taproot 交易在花费之前看起来完全相同，使得无法区分：
- 简单的单签名支付
- 复杂的多方合约  
- Lightning Network 操作
- 企业资金库交易

**效率提升**： 
- 更小的交易大小（64 字节签名）
- 更快的验证（单签名检查）
- 减少区块链膨胀（未使用的条件保持私有）

**协作激励**：Taproot 使经济激励与技术优化对齐——协作成为最有效的选择。

随着密钥调整奠定密码学基础，下一步是探索任意智能合约条件如何在 Merkle 树内紧凑承诺——在揭示之前保持不可见。

在下一章中，我们将探索 Merkle 树如何在统一外观背后组织复杂的脚本条件，展示无限的花费条件如何被承诺和证明，而不揭示未使用的替代方案。
